In [1]:
from langdetect import detect
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import datetime

In [2]:
#Function for sorting tf_idf in descending order
from scipy.sparse import coo_matrix
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [3]:
def ExtractKeyWords(text):
    #generate tf-idf for the given text
    tf_idf_vector=tfidf_transformer.transform(text)
    
    
    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())
    #extract only the top n; n here is 3
    keywords=extract_topn_from_vector(feature_names,sorted_items,3)
 
    # now print the results
    sentence = ""
    for k in keywords:
        sentence+=(" "+k)
    return sentence

In [4]:
l = ["../data/qald-3-test.csv","../data/qald-4-test.csv","../data/qald-5-test.csv","../data/qald-6-test.csv","../data/qald-7-test.csv","../data/qald-8-test.csv"]
times=[]
scores=[]

for d in l:
    df_test = pd.read_csv(d)
    df_test=df_test.dropna(subset=['questions'])
    vectorizer = CountVectorizer()
    X_train = vectorizer.fit_transform(df_test['questions'])
    tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
    X = tfidf_transformer.fit_transform(X_train)
    # get feature names
    feature_names=vectorizer.get_feature_names()
    new_set = []
    for text in X:
        new_set.append(ExtractKeyWords(text))
    f_prediction = []

    a = datetime.datetime.now()
    for sentence in new_set:
        f_prediction.append(detect(sentence))
    b = datetime.datetime.now()
    c = b-a
    times.append(c/len(f_prediction))
    scores.append(accuracy_score(df_test['lang'], f_prediction))


In [5]:
scores

[0.7188552188552189,
 0.8057142857142857,
 0.7574850299401198,
 0.75625,
 0.7674418604651163,
 0.6097560975609756]

In [6]:
times

[datetime.timedelta(microseconds=8562),
 datetime.timedelta(microseconds=7352),
 datetime.timedelta(microseconds=7645),
 datetime.timedelta(microseconds=7282),
 datetime.timedelta(microseconds=8192),
 datetime.timedelta(microseconds=9649)]

In [7]:
# total times spent and accuracy and rows
# q3 0.7188552188552189
# q4 0.8057142857142857
# q5 0.7365269461077845
# q6 0.7625
# q7 0.7674418604651163
# q8 0.5853658536585366

In [8]:
accuracy_score(df_test['lang'], f_prediction)

0.6097560975609756

In [9]:
# # Create a dataset by keywords
# output = pd.DataFrame({'questions': new_set, 'lang': df['lang']})
# output.to_csv('keyword.csv', index=False)

In [10]:
# out = []
# for w in new_set:
#     out.append(w+'\n\n')

# with open("key_word_test.txt","w") as f:
#     f.write("".join(out))